# Mission 2 패션 스타일 선호 여부 예측

# **Mission 2.1**

## 2.1.1. 라이브러리 Import

In [15]:
import os
import pandas as pd
from collections import defaultdict
from IPython.display import display  # Jupyter 환경에서 스타일링된 DataFrame을 출력하기 위한 display 함수

## 2.1.2 이미지ID 추출 함수

In [16]:
# 이미지ID 추출 함수
def extract_image_id(filename, is_image=True):
    parts = filename.split('_')
    return parts[1]  # 이미지ID는 두 번째 부분에 위치

## 2.1.3. 성별 및 스타일 통계 수집 함수

In [10]:
# 성별 & 스타일에 대한 통계를 수집하는 함수
def collect_statistics(image_dir, label_dir, chunk_size=5000):
    # 이미지 파일의 이름을 수집
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    # 이미지ID 추출 (이미지 파일 이름에서 이미지ID 부분만 추출)
    image_ids = set(f.split('_')[1] for f in image_files)

    # 성별 & 스타일에 대한 통계 저장 딕셔너리 (성별 -> 스타일 -> 설문ID 수)
    stats = defaultdict(lambda: defaultdict(int))

    # 한 번에 처리할 label 파일 개수를 설정
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.json')]

    # chunk 단위로 라벨 파일을 순회하면서 유효한 이미지ID에 대해 통계 수집
    for i in range(0, len(label_files), chunk_size):
        chunked_label_files = label_files[i:i + chunk_size]  # 청크 단위로 처리

        for label_file in chunked_label_files:
            # 라벨 파일 이름에서 이미지ID, 성별, 스타일 추출
            parts = label_file.split('_')
            image_id = parts[1]
            style = parts[3]
            gender = '여성' if parts[4] == 'W' else '남성'
            survey_id = parts[5].split('.')[0]  # 설문ID 추출

            # 이미지ID가 유효한 경우만 통계 수집
            if image_id in image_ids:
                stats[gender][style] += 1

 # 통계를 담을 리스트 생성
    data = []
    for gender, style_data in stats.items():
        for style, count in style_data.items():
            data.append([gender, style, count])

    # pandas 데이터프레임으로 변환
    df_stats = pd.DataFrame(data, columns=['성별', '스타일', '설문ID수'])

    # 성별 중복 값 제거 (빈 칸으로 남김)
    df_stats['성별'] = df_stats['성별'].mask(df_stats['성별'].duplicated(), '')

    return df_stats

## 2.1.5. 경로설정

In [17]:
# 경로 설정
train_image_dir = 'C:/training_image'  # Training 이미지 파일들이 저장된 경로
train_label_dir = 'C:/training_label'  # Training 라벨 파일들이 저장된 경로
val_image_dir = 'C:/validation_image'  # Validation 이미지 파일들이 저장된 경로
val_label_dir = 'C:/validation_label'  # Validation 라벨 파일들이 저장된 경로

## 2.1.6. Training 및 Validation 데이터셋 통계 수집

In [18]:
# training 데이터셋 통계 수집
train_stats_df = collect_statistics(train_image_dir, train_label_dir)

# validation 데이터셋 통계 수집
val_stats_df = collect_statistics(val_image_dir, val_label_dir)

## 2.1.7. 통계 결과 출력 함수

In [19]:
# 통계 결과 출력 함수
def display_statistics(df, dataset_name):
    print(f"\n{dataset_name} 데이터 통계")
    # pandas 스타일링 적용
    styled_df = df.style.set_properties(**{
        'border': '1px solid black',  # 셀 테두리
        'text-align': 'center',       # 텍스트 가운데 정렬
    }).hide(axis='index')  # 인덱스 숨김

    display(styled_df)  # display 함수를 사용하여 출력


# training 데이터셋 통계 출력
display_statistics(train_stats_df, "Training")

# validation 데이터셋 통계 출력
display_statistics(val_stats_df, "Validation")


Training 데이터 통계


성별,스타일,설문ID수
남성,hiphop,1321
,normcore,1096
,sportivecasual,1657
,ivy,1719
,mods,1615
,metrosexual,1226
,bold,1144
,hippie,1571
여성,ecology,295
,popart,195



Validation 데이터 통계


성별,스타일,설문ID수
여성,popart,38
,classic,117
,sportivecasual,278
,normcore,50
,hiphop,32
,lounge,20
,ecology,70
,minimal,171
,cityglam,65
,powersuit,160


# **Mission2.2**

# 2.2.1 모듈 임포트

In [20]:
import os
import json
import pandas as pd
from collections import Counter
from IPython.display import display

# 2.2.2 데이터 추출 함수 정의

In [21]:
# 이미지ID 추출 함수 (이미지와 JSON에서 동일한 부분을 추출)
def extract_base_image_id(filename):         # 파일 이름으로부터 고유 ID를 추출하는 함수 정의
    parts = filename.split('_')              # 파일 이름을 '_'(언더바) 기준으로 분할하여 리스트로 저장  
    return '_'.join(parts[1])  # 리스트의 두 번째 요소를 반환하여 ID를 생성 (예: W_02230_60_space_W에서 "02230") 

# json 파일에서 R_id와 스타일 선호 여부(Q5)를 추출하는 함수
def extract_r_id_and_preference(json_file):  # JSON 파일에서 R_id와 선호 여부를 추출하는 함수 정의
    try:
        with open(json_file, 'r', encoding='utf-8') as f: # JSON 파일을 UTF-8 인코딩으로 읽기 모드에서 열기
            data = json.load(f) # JSON 데이터를 파이썬의 딕셔너리 형태로 변환
        r_id = data['user']['R_id'] # 데이터에서 사용자 ID(R_id) 추출
        preference = data['item']['survey']['Q5']  # 선호 여부 추출 (1: 비선호, 2: 선호)
        return r_id, preference # 추출한 R_id와 선호 여부를 반환
    except (OSError, IOError, json.JSONDecodeError): # 파일 열기 오류, 파일 입출력 오류, JSON 디코딩 오류가 발생할 경우
        return None, None  # 문제가 있으면 None 반환

# 2.2.3 파일 필터링 함수 정의

In [22]:
# 유효한 JSON 파일 필터링 함수
def find_valid_json_files(image_dir, label_dir):
    # 이미지 파일의 ID 수집
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    image_ids = set(extract_base_image_id(f) for f in image_files)

    # 라벨 파일 수집
    json_files = [f for f in os.listdir(label_dir) if f.endswith('.json')]

    # 유효한 JSON 파일 필터링
    valid_json_files = [f for f in json_files if extract_base_image_id(f) in image_ids]
    
    return valid_json_files, image_files

# 2.2.4 공통된 R_id 추출 함수 정의

In [23]:
# 공통된 R_id 추출 함수 (Training과 Validation에서 선호와 비선호 모두 투표한 R_id만 추출)
def get_common_r_ids(train_label_dir, val_label_dir, valid_train_files, valid_val_files):
    train_r_ids = Counter()
    val_r_ids = Counter()

    # Training의 유효한 JSON 파일에서 R_id 추출
    for json_file in valid_train_files:
        json_path = os.path.join(train_label_dir, json_file)
        r_id, _ = extract_r_id_and_preference(json_path)
        if r_id:
            train_r_ids[r_id] += 1

    # Validation의 유효한 JSON 파일에서 R_id 추출
    for json_file in valid_val_files:
        json_path = os.path.join(val_label_dir, json_file)
        r_id, _ = extract_r_id_and_preference(json_path)
        if r_id:
            val_r_ids[r_id] += 1

    # Training과 Validation에 모두 응답한 R_id만 추출
    common_r_ids = {r_id for r_id in train_r_ids if r_id in val_r_ids}

    return common_r_ids, train_r_ids, val_r_ids

# 2.2.5 상위 100명의 응답자 추출 함수 정의

In [24]:
# 상위 100명의 응답자 추출 함수 (응답 빈도 기준)
def get_top_100_r_ids(common_r_ids, train_r_ids, val_r_ids):
    # 응답 빈도를 합산
    combined_r_ids = {r_id: train_r_ids[r_id] + val_r_ids[r_id] for r_id in common_r_ids}

    # 상위 100명 추출
    top_100_r_ids = sorted(combined_r_ids, key=combined_r_ids.get, reverse=True)[:100]
    
    return top_100_r_ids

# 2.2.6 R_id 테이블 생성 함수 정의

In [25]:
# 모든 R_id에 대한 스타일 통계 작성
def create_r_id_table(r_ids, valid_train_files, valid_val_files, train_label_dir, val_label_dir, train_images, val_images):
    data = []
    for r_id in r_ids:
        train_preference = {"선호": [], "비선호": []}
        val_preference = {"선호": [], "비선호": []}

        # Training JSON 파일에서 선호와 비선호 구분
        for train_file in valid_train_files:
            json_path = os.path.join(train_label_dir, train_file)
            train_r_id, preference = extract_r_id_and_preference(json_path)
            if train_r_id == r_id:
                image_filename = get_image_filename(train_file, train_images)
                if preference == 1:
                    train_preference["비선호"].append(image_filename)
                elif preference == 2:
                    train_preference["선호"].append(image_filename)

        # Validation JSON 파일에서 선호와 비선호 구분
        for val_file in valid_val_files:
            json_path = os.path.join(val_label_dir, val_file)
            val_r_id, preference = extract_r_id_and_preference(json_path)
            if val_r_id == r_id:
                image_filename = get_image_filename(val_file, val_images)
                if preference == 1:
                    val_preference["비선호"].append(image_filename)
                elif preference == 2:
                    val_preference["선호"].append(image_filename)

        data.append({
            "응답자ID": r_id,
            "Training 스타일 선호": ', '.join(train_preference["선호"]),
            "Training 스타일 비선호": ', '.join(train_preference["비선호"]),
            "Validation 스타일 선호": ', '.join(val_preference["선호"]),
            "Validation 스타일 비선호": ', '.join(val_preference["비선호"])
        })

    df_table = pd.DataFrame(data)
    return df_table

# 2.2.7 보조 함수 정의

In [26]:
# 이미지 파일명 찾기 함수
def get_image_filename(json_filename, image_files):
    base_id = extract_base_image_id(json_filename)
    for img_file in image_files:
        if extract_base_image_id(img_file) == base_id:
            return img_file
    return None

# 스타일링된 테이블을 출력하는 함수
def display_styled_table(df):
    styled_df = df.style.set_properties(**{
        'border': '1px solid black',
        'text-align': 'center'
    }).hide(axis='index').set_table_styles([{
        'selector': 'th',
        'props': [('border', '1px solid black'), ('text-align', 'center')]
    }])
    
    display(styled_df)

# 2.2.8 공통된 R_id 추출 후 응답 빈도 순으로 상위 100명 추출

In [27]:
# 경로 설정
train_image_dir = 'C:/training_image'
train_label_dir = 'C:/training_label'
val_image_dir = 'C:/validation_image'
val_label_dir = 'C:/validation_label'

# 1. Training과 Validation에서 유효한 JSON 파일 찾기
valid_train_files, train_images = find_valid_json_files(train_image_dir, train_label_dir)
valid_val_files, val_images = find_valid_json_files(val_image_dir, val_label_dir)

# 2. Training과 Validation에 모두 응답한 공통된 R_id 추출
common_r_ids, train_r_ids, val_r_ids = get_common_r_ids(train_label_dir, val_label_dir, valid_train_files, valid_val_files)

# 3. 응답 빈도에 따라 상위 100명의 R_id 추출
top_100_r_ids = get_top_100_r_ids(common_r_ids, train_r_ids, val_r_ids)

# 4. 상위 100명의 R_id에 대한 테이블 생성
df_r_id_table = create_r_id_table(top_100_r_ids, valid_train_files, valid_val_files, train_label_dir, val_label_dir, train_images, val_images)

# 5. 테이블 출력 (상위 100명)
display_styled_table(df_r_id_table)

# 6. 결과를 CSV로 저장
df_r_id_table.to_csv('C:/dataFrame/top_100_r_id_table.csv', index=False)

응답자ID,Training 스타일 선호,Training 스타일 비선호,Validation 스타일 선호,Validation 스타일 비선호
64747,"W_03194_50_classic_W.jpg, W_04972_90_kitsch_W.jpg, W_05628_00_cityglam_W.jpg, W_07894_00_cityglam_W.jpg, W_14901_90_kitsch_W.jpg, W_20598_70_military_W.jpg, W_21223_80_powersuit_W.jpg, W_21483_19_genderless_W.jpg, W_22057_19_genderless_W.jpg, W_22510_80_powersuit_W.jpg, W_29783_10_sportivecasual_W.jpg, W_30399_19_genderless_W.jpg, W_30434_60_minimal_W.jpg, W_30454_60_minimal_W.jpg, W_31416_70_hippie_W.jpg, W_34636_00_oriental_W.jpg, W_35674_60_minimal_W.jpg, W_37160_70_punk_W.jpg, W_38421_10_athleisure_W.jpg, W_38629_80_powersuit_W.jpg, W_39725_19_normcore_W.jpg, W_44330_10_sportivecasual_W.jpg, W_46907_80_powersuit_W.jpg, W_48628_19_genderless_W.jpg","W_02247_50_classic_W.jpg, W_02498_50_feminine_W.jpg, W_03643_00_cityglam_W.jpg, W_11610_90_grunge_W.jpg, W_13904_50_feminine_W.jpg, W_14102_50_feminine_W.jpg, W_18951_50_feminine_W.jpg, W_27828_60_minimal_W.jpg, W_33026_90_hiphop_W.jpg, W_34024_10_sportivecasual_W.jpg, W_34573_10_sportivecasual_W.jpg, W_36644_00_oriental_W.jpg, W_37025_19_lounge_W.jpg, W_38771_10_sportivecasual_W.jpg, W_40690_80_bodyconscious_W.jpg, W_40876_70_punk_W.jpg, W_41633_60_space_W.jpg, W_42595_60_popart_W.jpg, W_44386_80_powersuit_W.jpg, W_47169_70_hippie_W.jpg, W_48378_90_grunge_W.jpg","W_05628_00_cityglam_W.jpg, W_20598_70_military_W.jpg, W_22510_80_powersuit_W.jpg, W_30988_90_kitsch_W.jpg, W_37491_70_military_W.jpg, W_38588_19_genderless_W.jpg, W_39164_00_oriental_W.jpg, W_44330_10_sportivecasual_W.jpg, W_46907_80_powersuit_W.jpg","W_02498_50_feminine_W.jpg, W_11610_90_grunge_W.jpg, W_14102_50_feminine_W.jpg, W_27828_60_minimal_W.jpg, W_34024_10_sportivecasual_W.jpg, W_47169_70_hippie_W.jpg"
63405,"W_00023_60_mods_M.jpg, W_00555_50_ivy_M.jpg, W_01670_10_sportivecasual_M.jpg, W_01853_60_mods_M.jpg, W_02890_19_normcore_M.jpg, W_02931_00_metrosexual_M.jpg, W_04206_60_mods_M.jpg, W_04522_50_ivy_M.jpg, W_04684_90_hiphop_M.jpg, W_06682_00_metrosexual_M.jpg, W_06785_50_ivy_M.jpg, W_06812_50_ivy_M.jpg, W_07021_90_hiphop_M.jpg, W_07198_10_sportivecasual_M.jpg, W_12551_19_normcore_M.jpg, W_12814_19_normcore_M.jpg, W_15294_50_ivy_M.jpg, W_15400_60_mods_M.jpg, W_16084_80_bold_M.jpg, W_17510_19_normcore_M.jpg, W_17849_00_metrosexual_M.jpg, W_17909_60_mods_M.jpg","W_00530_10_sportivecasual_M.jpg, W_02714_00_metrosexual_M.jpg, W_06691_10_sportivecasual_M.jpg, W_12383_80_bold_M.jpg, W_12443_90_hiphop_M.jpg, W_12456_10_sportivecasual_M.jpg, W_12904_50_ivy_M.jpg, W_15134_80_bold_M.jpg, W_15140_80_bold_M.jpg, W_15471_10_sportivecasual_M.jpg, W_15472_70_hippie_M.jpg, W_15517_70_hippie_M.jpg, W_15782_70_hippie_M.jpg, W_16501_70_hippie_M.jpg, W_16736_00_metrosexual_M.jpg, W_16755_00_metrosexual_M.jpg, W_16919_19_normcore_M.jpg, W_17108_19_normcore_M.jpg, W_17219_70_hippie_M.jpg, W_17443_90_hiphop_M.jpg, W_17454_80_bold_M.jpg, W_17599_19_normcore_M.jpg","W_01853_60_mods_M.jpg, W_02677_60_mods_M.jpg, W_02879_90_hiphop_M.jpg, W_04522_50_ivy_M.jpg, W_04684_90_hiphop_M.jpg, W_06860_19_normcore_M.jpg, W_15294_50_ivy_M.jpg","W_07187_70_hippie_M.jpg, W_12304_80_bold_M.jpg, W_12904_50_ivy_M.jpg, W_15140_80_bold_M.jpg, W_16501_70_hippie_M.jpg, W_16755_00_metrosexual_M.jpg, W_17443_90_hiphop_M.jpg"
64346,"W_00047_70_hippie_M.jpg, W_00856_10_sportivecasual_M.jpg, W_02754_70_hippie_M.jpg, W_09154_50_ivy_M.jpg, W_09233_60_mods_M.jpg, W_09239_19_normcore_M.jpg, W_09289_19_normcore_M.jpg, W_09856_10_sportivecasual_M.jpg, W_12155_80_bold_M.jpg, W_12527_70_hippie_M.jpg, W_16233_80_bold_M.jpg, W_16777_60_mods_M.jpg, W_21401_00_metrosexual_M.jpg, W_23905_60_mods_M.jpg, W_23920_19_normcore_M.jpg, W_23922_50_ivy_M.jpg, W_24103_50_ivy_M.jpg, W_24268_00_metrosexual_M.jpg, W_24874_80_bold_M.jpg, W_24977_70_hippie_M.jpg, W_26175_50_ivy_M.jpg, W_28693_10_sportivecasual_M.jpg, W_29918_19_normcore_M.jpg, W_29990_90_hiphop_M.jpg, W_30040_60_mods_M.jpg, W_49510_00_metrosexual_M.jpg","W_02678_50_ivy_M.jpg, W_07112_90_hiphop_M.jpg, W_12268_80_bold_M.jpg, W_15769_00_metros